# Verse Interpreter development

Essential installs:
- `pip3 install .....`


This version is used to test the first steps for the verse interpreter. The final version will be used as a full python file.

In [5]:
import string
import math
from enum import Enum

# Error Class

In [6]:
class ErrorType(Enum):
    SyntaxError = 'Wrong Syntax at'
    SemanticError = 'Wrong Semantics at'
    UnkownError = 'Operation Failure'

# Logger

In [7]:
class Logger:
    def __init__(self):{}
        
    def __log__(self, string:str):{}

    def __log_error__(self,string:str, type:ErrorType):{}

class Console_Logger(Logger):

    def __log__(self, string:str):
        print(string)

    def __log_error__(self,string:str, type:ErrorType):       
        print("ERROR| " + type.value + ": " + string)

## Setting Token Enum for testing purpose

In [8]:
class TokenTypes(Enum):
    # Data
    INTEGER = int
    IDENTIFIER = string #Names/Variables
    INT_TYPE = "int"
    TUPLE_TYPE = "tuple"
    ARRAY_TYPE = "array"
    # Aritmetics
    PLUS = "+"
    MINUS = "-"
    MULTIPLY = "*"
    DIVIDE = "/"
    GREATER = ">"
    GREATEREQ = ">="
    LOWER = "<"
    LOWEREQ = "<="
    CHOICE = "|"
    # Mehtods
    FOR = "for"
    DO = "do"
    IF = "if"
    THEN = "then"
    ELSE = "else"
    # Else
    EOF = None
    COLON = ":"
    COMMA=","
    SEMICOLON =";"
    BINDING =":="
    LBRACKET = "("
    RBRACKET = ")"
    SBL = "["
    SBR = "]"
    CBL = "{"
    CBR = "}"
    EQUAL = "="
    SCOPE = ":"
    DOT = "."

In [9]:
list(TokenTypes)

[<TokenTypes.INTEGER: <class 'int'>>,
 <TokenTypes.IDENTIFIER: <module 'string' from '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/string.py'>>,
 <TokenTypes.INT_TYPE: 'int'>,
 <TokenTypes.TUPLE_TYPE: 'tuple'>,
 <TokenTypes.ARRAY_TYPE: 'array'>,
 <TokenTypes.PLUS: '+'>,
 <TokenTypes.MINUS: '-'>,
 <TokenTypes.MULTIPLY: '*'>,
 <TokenTypes.DIVIDE: '/'>,
 <TokenTypes.GREATER: '>'>,
 <TokenTypes.GREATEREQ: '>='>,
 <TokenTypes.LOWER: '<'>,
 <TokenTypes.LOWEREQ: '<='>,
 <TokenTypes.CHOICE: '|'>,
 <TokenTypes.FOR: 'for'>,
 <TokenTypes.DO: 'do'>,
 <TokenTypes.IF: 'if'>,
 <TokenTypes.THEN: 'then'>,
 <TokenTypes.ELSE: 'else'>,
 <TokenTypes.EOF: None>,
 <TokenTypes.COLON: ':'>,
 <TokenTypes.COMMA: ','>,
 <TokenTypes.SEMICOLON: ';'>,
 <TokenTypes.BINDING: ':='>,
 <TokenTypes.LBRACKET: '('>,
 <TokenTypes.RBRACKET: ')'>,
 <TokenTypes.SBL: '['>,
 <TokenTypes.SBR: ']'>,
 <TokenTypes.CBL: '{'>,
 <TokenTypes.CBR: '}'>,
 <TokenTypes.EQUAL: '='>,
 <TokenTypes.DOT: '.'>]

In [10]:
class Token:
    def __init__(self, type: TokenTypes, value) -> None:
        self.type = type
        self.value = value
    
    def __info__(self):
         return "{}: {}".format(self.type, self.value)

## Lexer

In [11]:
class lexicon:
    def __init__(self, input: string):
        self.input = input
        self.index = 0
        self.current_char = self.input[self.index]
    
    # moves the pointer a character forward
    def forward(self) -> None:
        self.index += 1

        # checks if index is out of range
        if (self.index >= len(self.input)):
            self.current_char = None
            return
        
        self.current_char = self.input[self.index]
    
    def backward(self) -> None:
        self.index -= 1

        # checks if index is out of range
        if self.index < 0:
            self.current_char = None
            return
        
        self.current_char = self.input[self.index]
    
    def get_int(self) -> int:
        if self.index >= len(self.input):
            return None
        
        result = self.input[self.index]

        # checks if there are multiple digits
        while True:
            self.forward()

            if self.index < len(self.input) and self.input[self.index] != None and self.input[self.index].isnumeric():
                result += self.input[self.index]
            else:
                self.backward()
                break

        return int(result)
    
    def get_var(self) -> string:
        if self.index >= len(self.input):
            return None
        
        result = self.input[self.index]

        # checks if there is a longer variable name
        while True:
            self.forward()

            if self.index < len(self.input) and self.input[self.index] != None and self.input[self.index].isalpha():
                result += self.input[self.index]
            else:
                self.backward()
                break
        
        return result
    
    def get_binding(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.input[self.index]
        self.forward()
        
        if self.index < len(self.input) and self.input[self.index] != None:
                result += self.input[self.index]

        match result:
            case TokenTypes.BINDING.value:
                return Token(TokenTypes.BINDING, TokenTypes.BINDING.value)
        
        self.backward()
        return Token(TokenTypes.COLON, TokenTypes.COLON.value)
    
    def get_greater_eq(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.input[self.index]
        self.forward()
        
        if self.index < len(self.input) and self.input[self.index] != None:
                result += self.input[self.index]

        match result:
            case TokenTypes.GREATEREQ.value:
                return Token(TokenTypes.GREATEREQ, TokenTypes.GREATEREQ.value)
        
        self.backward()
        return Token(TokenTypes.GREATER, TokenTypes.GREATER.value)
    
    def get_lower_eq(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.input[self.index]
        self.forward()
        
        if self.index < len(self.input) and self.input[self.index] != None:
                result += self.input[self.index]

        match result:
            case TokenTypes.LOWEREQ.value:
                return Token(TokenTypes.LOWEREQ, TokenTypes.LOWEREQ.value)
        
        self.backward()
        return Token(TokenTypes.LOWER, TokenTypes.LOWER.value)
    
    def get_token(self, char: string) -> Token:
        token = self.check_for_tokentypes(char)

        if token.type != TokenTypes.EOF:
            return token
        
        if char == None:
            return token
            
        # skip spaces.
        if char == ' ':
            self.forward()
            return self.get_token(self.current_char)
        
        if char == TokenTypes.COLON:
            return self.get_next(TokenTypes.BINDING.value)

        # checks if the current character is a number.
        if char.isnumeric():
            result = self.get_int()
            return Token(TokenTypes.INTEGER, result)
        
        if char.isalpha():
            result = self.get_var()
            token = self.check_for_tokentypes(result)
                  
        return token

    def check_for_tokentypes(self, char: string) -> Token:
        # checks if the current character is a supported token type.
        match char:
            case TokenTypes.INTEGER.value:
                return Token(TokenTypes.INTEGER, TokenTypes.INTEGER.value)
            case TokenTypes.IDENTIFIER.value:
                return Token(TokenTypes.IDENTIFIER, TokenTypes.IDENTIFIER.value)
            case TokenTypes.INT_TYPE.value:
                return Token(TokenTypes.INT_TYPE, TokenTypes.INT_TYPE.value)
            case TokenTypes.TUPLE_TYPE.value:
                return Token(TokenTypes.TUPLE_TYPE, TokenTypes.TUPLE_TYPE.value)
            case TokenTypes.ARRAY_TYPE.value:
                return Token(TokenTypes.ARRAY_TYPE, TokenTypes.ARRAY_TYPE.value)
            case TokenTypes.PLUS.value:
                return Token(TokenTypes.PLUS, TokenTypes.PLUS.value)
            case TokenTypes.MINUS.value:
                return Token(TokenTypes.MINUS, TokenTypes.MINUS.value)
            case TokenTypes.MULTIPLY.value:
                return Token(TokenTypes.MULTIPLY, TokenTypes.MULTIPLY.value)
            case TokenTypes.DIVIDE.value:
                return Token(TokenTypes.DIVIDE, TokenTypes.DIVIDE.value)
            case TokenTypes.GREATER.value:
                return self.get_greater_eq()
            case TokenTypes.GREATEREQ.value:
                return Token(TokenTypes.GREATEREQ, TokenTypes.GREATEREQ.value)
            case TokenTypes.LOWER.value:
                return self.get_lower_eq()
            case TokenTypes.LOWEREQ.value:
                return Token(TokenTypes.LOWEREQ, TokenTypes.LOWEREQ.value)
            case TokenTypes.CHOICE.value:
                return Token(TokenTypes.CHOICE, TokenTypes.CHOICE.value)
            case TokenTypes.FOR.value:
                return Token(TokenTypes.FOR, TokenTypes.FOR.value)
            case TokenTypes.DO.value:
                return Token(TokenTypes.DO, TokenTypes.DO.value)
            case TokenTypes.IF.value:
                return Token(TokenTypes.IF, TokenTypes.IF.value)
            case TokenTypes.THEN.value:
                return Token(TokenTypes.THEN, TokenTypes.THEN.value)
            case TokenTypes.ELSE.value:
                return Token(TokenTypes.ELSE, TokenTypes.ELSE.value)
            case TokenTypes.EOF.value:
                return Token(TokenTypes.EOF, TokenTypes.EOF.value)
            case TokenTypes.COLON.value:
                return self.get_next()
            case TokenTypes.COMMA.value:
                return Token(TokenTypes.COMMA, TokenTypes.COMMA.value)
            case TokenTypes.SEMICOLON.value:
                return Token(TokenTypes.SEMICOLON, TokenTypes.SEMICOLON.value)
            case TokenTypes.BINDING.value:
                return Token(TokenTypes.BINDING, TokenTypes.BINDING.value)
            case TokenTypes.LBRACKET.value:
                return Token(TokenTypes.LBRACKET, TokenTypes.LBRACKET.value)
            case TokenTypes.RBRACKET.value:
                return Token(TokenTypes.RBRACKET, TokenTypes.RBRACKET.value)
            case TokenTypes.RBRACKET.value:
                return Token(TokenTypes.RBRACKET, TokenTypes.RBRACKET.value)
            case TokenTypes.SBR.value:
                return Token(TokenTypes.SBR, TokenTypes.SBR.value)
            case TokenTypes.CBL.value:
                return Token(TokenTypes.CBL, TokenTypes.CBL.value)
            case TokenTypes.CBR.value:
                return Token(TokenTypes.CBR, TokenTypes.CBR.value)
            case TokenTypes.EQUAL.value:
                return Token(TokenTypes.EQUAL, TokenTypes.EQUAL.value)
            case TokenTypes.SCOPE.value:
                return Token(TokenTypes.SCOPE, TokenTypes.SCOPE.value)
            case TokenTypes.DOT.value:
                return Token(TokenTypes.DOT, TokenTypes.DOT.value)
            case _:
                return Token(TokenTypes.EOF, None)

## Testing Lexer Types

In [12]:
lexer = lexicon(">=")

while lexer.current_char is not None:
    token = lexer.get_token(lexer.current_char)
    print(str(token.value) + " is of the tokentype: " + str(token.type))
    lexer.forward()

>= is of the tokentype: TokenTypes.GREATEREQ


## ParserNode

In [13]:
#Class that takes a parsed node, containes information if node could have been parsed
class ParsedNode:
    def __init__(self, node, hasSyntaxError:bool ):
        self.node = node
        self.hasSyntaxError = hasSyntaxError

## Syntax Tree Nodes

In [46]:
class BaseNode:
    def __init__(self, token) -> None:
        self.token = token
    
    def visit(self, node):
        if isinstance(node, ProgramNode):
                return self.visit_programNode(node)
        elif isinstance(node, BlockNode):
                return self.visit_blockNode(node)
        elif isinstance(node, ScopeNode):
                return self.visit_scopeNode(node)
        elif isinstance(node, OperatorNode):
                return self.visit_operatorNode(node)
        elif isinstance(node, NumberNode):
                return self.visit_numberNode(node)
        elif isinstance(node, UnaryNode):
                return self.visit_unaryNode(node)
        elif isinstance(node, ParsedNode):
             return self.visit(node.node)
        elif isinstance(node, ForNode):
            return self.visit_forNode(node)
    
    def visit_programNode(self, node):
        return self.visit(node.node)

    def visit_blockNode(self, node):
        for n in node.nodes:
             return self.visit(n)

    def visit_scopeNode(self, node):
        pass

    def visit_unaryNode(self, node):
        pass
        
    def visit_operatorNode(self, node):
        match node.token.type:
            case TokenTypes.DIVIDE:
                return self.visit(node.leftNode) // self.visit(node.rightNode)
            case TokenTypes.MULTIPLY:
                return self.visit(node.leftNode) * self.visit(node.rightNode)
            case TokenTypes.PLUS:
                return self.visit(node.leftNode) + self.visit(node.rightNode)
            case TokenTypes.MINUS:
                return self.visit(node.leftNode) - self.visit(node.rightNode)   
            case TokenTypes.EQUAL:
                if(self.visit(node.leftNode) == self.visit(node.rightNode)):
                    return self.visit(node.leftNode)
                return ""
            case TokenTypes.GREATER:
                if(self.visit(node.leftNode) > self.visit(node.rightNode)):
                    return self.visit(node.leftNode)
                return ""
            case TokenTypes.LOWER:
                 if(self.visit(node.leftNode) < self.visit(node.rightNode)):
                    return self.visit(node.leftNode)
                 return ""     
            case TokenTypes.DOT:
                return [i for i in range(self.visit(node.leftNode), self.visit(node.rightNode)+1)]

    def visit_numberNode(self, node):
        return node.value
    
    def visit_forNode(self, node):
        pass

class BlockNode(BaseNode):
    def __init__(self, nodes:list[BaseNode]) -> None:
        self.nodes = nodes

class ProgramNode(BaseNode):
    def __init__(self, node:BlockNode) -> None:
        self.node = node


class ScopeNode(BaseNode):
    def __init__(self,token:Token, nodes:list[BaseNode]) -> None:
        super().__init__(token)
        self.nodes = nodes

class NumberNode(BaseNode):
    def __init__(self, token:Token) -> None:
        super().__init__(token)
        self.value = token.value

class OperatorNode(BaseNode):
    def __init__(self, token:Token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

class StatementNode(BaseNode):
     def __init__(self, token:Token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

class UnaryNode(BaseNode):
     def __init__(self, token:Token, node) -> None:
        super().__init__(token)
        self.node = node

class ScopeNode(BaseNode):
    def __init__(self, token:Token, nodes:list[BaseNode], type) -> None: #Change into Variable/IdentifierNode
        super().__init__(token)
        self.nodes = nodes
        self.type = type

class TypeNode(BaseNode):
    def __init__(self, token:Token, type) -> None: 
        super().__init__(token)
        self.type = type

class TypeNodeSequence(TypeNode):
    def __init__(self, token:Token, nodes:list[TypeNode], type) -> None: 
        super().__init__(token)
        self.nodes = nodes
        self.type = type


class ArgumentsNode: #Doesnt need BaseNode since it doesn't have token
    def __init__(self, nodes:list[BaseNode]) -> None: 
        self.nodes = nodes

class FuncCallNode:
    def __init__(self,identifier:BaseNode, args:ArgumentsNode) -> None: #instead of BaseNode maybe IdentifierNode
        self.identifier = identifier
        self.args = args

class ForNode(BaseNode):
     def __init__(self, token:Token, nodes: list[BaseNode], condition: BaseNode, expr: BaseNode) -> None:
        super().__init__(token)
        self.nodes = nodes
        self.condition = condition
        self.expr = expr

class IfNode(BaseNode):
     def __init__(self, token:Token, if_node: BaseNode, then_node: BaseNode, else_node: BaseNode) -> None:
        super().__init__(token)
        self.if_node = if_node
        self.then_node = then_node
        self.else_node = else_node

In [50]:
class Parser:
    def __init__(self, lexer: lexicon):
       self.logger: Logger = Console_Logger()
       self.end = False
       self.lexer = lexer
       self.current_token = lexer.get_token(self.lexer.current_char)
       

    def parse(self) -> BaseNode:     
        node = self.program()

        if node.hasSyntaxError:
            self.logger.__log_error__("it appears there was a problem", ErrorType.SyntaxError)
        return node.node
       
    #####################################
    # statements
    #####################################

    """
    Checks if the program has either statements or function calls.
    Rule => statement_list | func_decl (SEMI statement_list | func_decl)*?
    """
    def program(self) -> ParsedNode:
        node = self.block()

        # checks if start of the program is not a block but a function.
        if(node.hasSyntaxError == True):
            node = self.func_decl()
        
        # checks if the program is not a function either.
        if(node.hasSyntaxError == True):
            return ParsedNode(None, True)
        
        return ParsedNode(ProgramNode(node.node), False)

    """
    Checks if the program has a list of statements.
    Rule => statement_list
    """
    def block(self) -> ParsedNode:
        nodes = self.statement_list()
        
        return ParsedNode(BlockNode(nodes), False)

    """
    Checks if there are one or more statements.
    Rule => statement (SEMI statement)*?   
    """
    def statement_list(self) -> list[BaseNode]:
        parsed_nodes = []
        parsed_nodes.append(self.statement())
        nodes = []

        while True:
            if(self.current_token.type == TokenTypes.SEMICOLON):
                self.forward()
                parsed_nodes.append(self.statement())
            else:
                for node in parsed_nodes:
                    if node.hasSyntaxError:
                        return [ParsedNode(None, True)]
                    else:
                        nodes.append(node)
                break

        return nodes

    """
    Checks if the current node is either an expression statemnts, if statement, for statement, a function call or an assignment statemnt.
    Rule => expr | if | for | func_call | assign_statement
    """
    def statement(self) -> ParsedNode:
        node: ParsedNode = self.expr()

        # checks if current node is not an expression.
        if(node.hasSyntaxError == True):
            node = self.if_statement()
        
        # checks if current node is not an if statement either.
        if(node.hasSyntaxError == True):
            node = self.for_loop()
        
        # checks if current node is not a for loop either.
        if(node.hasSyntaxError == True):
            node = self.assign_statement()

        return node
    
    """
    Checks if the statement assigns a something to an identifier
    Rule => binding_statement EQ (method_call|expr)          [=]
    """
    def assign_statement(self) -> ParsedNode:
        leftNode = self.binding_statement()
        token = self.current_token

        if token.type != TokenTypes.EQUAL:
            return ParsedNode(None, True)
        
        self.forward()

        # checks if the rightNode is an expression.
        rightNode = self.expr()

        # checks if the rightNode was not an expression and tries a method call.
        if rightNode.hasSyntaxError:
            rightNode = self.func_call()
        
        if leftNode.hasSyntaxError == False and rightNode.hasSyntaxError == False:
            return ParsedNode(StatementNode(token, leftNode, rightNode), False)
        
        return ParsedNode(None, True)


    """
    Checks if the statement binds a value to a scope
    Rule => scope_statement BINDING (method_call|expr)       [:=]
    """
    def binding_statement(self) -> ParsedNode:
        leftNode = self.scope_statement()
        token = self.current_token

        if token.type != TokenTypes.BINDING:
            return ParsedNode(None, True)
        
        self.forward()

        # checks if the rightNode is an expression.
        rightNode = self.expr()

        # checks if the rightNode was not an expression and tries a method call.
        if rightNode.hasSyntaxError:
            rightNode = self.func_call()
        
        if leftNode.hasSyntaxError == False and rightNode.hasSyntaxError == False:
            return ParsedNode(StatementNode(token, leftNode, rightNode), False)
    
        return ParsedNode(None, True)

    """
    Checks if the statement scopes a value to a type
    Rule => IDENTIFIER (,IDENTIFIER)*? COLON TYPE            [:]
    """
    def scope_statement(self) -> list[ParsedNode]:
        nodes = []
        nodes.append(self.identifier())

        while True:
            if(self.current_token.type == TokenTypes.COMMA):
                self.forward()
                nodes.append(self.identifier())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return ParsedNode(None, True)
                break
        
        if self.current_token.type != TokenTypes.COLON:
            return ParsedNode(None, True)
        
        self.forward
        type = self.type()

        if nodes[0].hasSyntaxerror == False and type.hasSyntaxError == False:
            return ParsedNode(ScopeNode(TokenTypes.COLON, nodes, type))
        
        return ParsedNode(None, True)
            

    """
    Checks for a function call.
    Rule => IDENTIFIER LB (func_call_args)? RB 
    """
    def func_call(self) -> ParsedNode:
        # RULE --> IDENTIFIER LB (func_call_param)? RB  NOT IMPLEMENTED
        node = self.identifier()

        if(self.current_token.type != TokenTypes.LBRACKET):
            return ParsedNode(None, True)
        
        self.forward()
        # checks if it is an empty function call.
        if self.current_token.type == TokenTypes.RBRACKET:
            return ParsedNode(FuncCallNode(node, None), False)

        params = self.func_call_args()
        for param in params:
            if param.hasSyntaxError:
                return ParsedNode(None, True)
            
        if(self.current_token.type == TokenTypes.RBRACKET):
            return ParsedNode(FuncCallNode(node, params), False)
            
        return node
        

    """
    Checks for the arguments of the function call.
    Rule => expr (COMMA expr)*?  
    """    
    def func_call_args(self) -> list[ParsedNode]:
        nodes = []
        nodes.append(self.expr())

        while True:
            if(self.current_token.type == TokenTypes.COMMA):
                self.forward()
                nodes.append(self.expr())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return [ParsedNode(None, True)]
                break
        
        return nodes


    """
    Checks for the declaration of a function.
    Rule => IDENTIFIER LB func_dec_param RB (COLON type)? BINDING block
           |IDENTIFIER BINDING LB nested_scope LAMBDA expr RB    
    """
    def func_decl(self) -> ParsedNode:
        # identifier = self.identifier()
         
        # if self.current_token.type != TokenTypes.LBRACKET:
        #     return ParsedNode(None, True)
        
        # self.forward()
        # params = self.func_decl_param()

        # if self.current_token.type != TokenTypes.RBRACKET:
        #     return ParsedNode(None, True)
        pass

    """
    Checks for the arguments of the function declaration.
    Rule => nested_scope
    """
    def func_decl_param(self) -> ParsedNode:
        pass

    """
    Checks if the statement is an if statement.
    Rule => IF LB expr RB THEN CBL block CBR ELSE CBL block CBR
          | IF LB expr RB THEN expr ELSE expr  
    """
    def if_statement(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.IF:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token != TokenTypes.LBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if_node = self.expr()
        if if_node.hasSyntaxError == True or self.current_token != TokenTypes.RBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token != TokenTypes.THEN:
            return ParsedNode(None, True)
        
        self.forward()
        then_node = self.then_statement()
        if then_node.hasSyntaxError == True or self.current_token != TokenTypes.ELSE:
            return ParsedNode(None, True)
        
        self.forward()
        else_node = self.else_statement()

        if else_node.hasSyntaxError:
            return ParsedNode(None, True)
        
        return ParsedNode(IfNode(token, if_node, then_node, else_node))
            

    """
    Checks the 'Then' part of the if-statement.
    Rule => expr | (CBL block CBR)
    """
    def then_statement(self) -> ParsedNode:
        pass


    """
    Checks the 'else' part of the if-statement.
    Rule => expr | (CBL block CBR)
    """
    def else_statement(self) -> ParsedNode:
        pass

    """
    Checks if the statement is a loop expression.
    Rule => FOR LCB (scope|expr) (condition)? (;expr)? RCB                     --> LCB/RCB means (LEFT/RIGHT) CURVY BRACKETS; FOR means for
        | FOR LCB expr (| expr)*?                                              --> returns Tuple
        | FOR LCB expr (--> CONTAINS DOT DOT) RCB                              --> List
        | FOR LB (scope|expr) (condition)? (;expr)? RB DO expr
    """
    def for_loop(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.FOR:
            return ParsedNode(None, True)
        
        self.forward()
        # checks if the for loop is defined with a curly bracket.
        if self.current_token.type == TokenTypes.CBL:
            self.forward()
            return self.for_loop_curly()
        elif self.current_token.type == TokenTypes.RBRACKET:
            self.forward()
            return self.for_loop_bracket()

        return ParsedNode(None, True)
            
    """
    Checks if the statement is a loop expression.
    Rule => FOR LCB (scope|expr) (;condition)? (;expr)? RCB                     --> LCB/RCB means (LEFT/RIGHT) CURVY BRACKETS; FOR means for
        | FOR LCB expr (| expr)*?                                              --> returns Tuple
        | FOR LCB expr (--> CONTAINS DOT DOT) RCB       
    """
    def for_loop_curly(self) -> ParsedNode:
        node = self.scope()
        nodes = []

        # checks if the loop content is not a scope but an expression.
        if node.hasSyntaxError == True:
            node = self.expr()
        
        # checks if the loop input is invalid.
        if node.hasSyntaxError:
            return ParsedNode(None, True)
        
        nodes.append(node)
        while True:
            if self.current_token.type == TokenTypes.SEMICOLON:
                self.forward()
                nodes.append(self.expression())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return ParsedNode(None, True)
                break
        
        if self.current_token.type != TokenTypes.CBR:
            return ParsedNode(None, True)
        
        return ParsedNode(ForNode(TokenTypes.FOR, nodes, None, None),False)
    
    def for_loop_bracket(self) -> ParsedNode:
        node = self.scope()
        nodes = []

        # checks if the loop content is not a scope but an expression.
        if node.hasSyntaxError == True:
            node = self.expr()
        
        # checks if the loop input is invalid.
        if node.hasSyntaxError:
            return ParsedNode(None, True)
        
        nodes.append(node)
        while True:
            if self.current_token.type == TokenTypes.SEMICOLON:
                self.forward()
                nodes.append(self.expression())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return ParsedNode(None, True)
                break
        
        if self.current_token.type != TokenTypes.RBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.DO:
            return ParsedNode(None, True)
        
        self.forward()
        do = self.expr()

        if do.hasSyntaxError:
            return ParsedNode(None, True)
        
        return ParsedNode(ForNode(token, nodes, do))


    """
    Checks if the statement is a nested scope.
    Rule =>  IDENTIFIER (,IDENTIFIER)*? COLON TYPE
    """
    def nested_scope(self) -> ParsedNode:
        nodes: list[ParsedNode] = []
        nodes.append(self.identifier())

        if nodes[0].hasSyntaxError:
            return ParsedNode(None, True)
        
        while True:
            if self.current_token.type == TokenTypes.COMMA:
                self.forward()
                nodes.append(self.identifier())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return ParsedNode(None, True)
                break
        
        if self.current_token != TokenTypes.COLON:
            return ParsedNode(None, True)
        
        self.forward()
        type = self.type()

        if type.hasSyntaxError == True:
            return ParsedNode(None, True)

        return ParsedNode(ScopeNode(TokenTypes.COLON, nodes, type))
        

    #####################################
    # expressions
    #####################################

    """
    Rule => operation DOT DOT operation
    """
    def expr(self) -> ParsedNode:         
        leftNode = self.operation()

        if leftNode.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type != TokenTypes.DOT:
            return leftNode
        
        self.forward()
        if self.current_token.type != TokenTypes.DOT:
            return ParsedNode(None, True)
        
        token = self.current_token
        self.forward()
        rightNode = self.operation()

        if rightNode.hasSyntaxError:
            return ParsedNode(None, True)

        return ParsedNode(OperatorNode(token, leftNode.node, rightNode.node), False)
    
    """
    This method checks if a token any of the following operations: =, <, >, <=, >=, |, +, -
    Since all of this operations have the same priority and same values output, it is not needed to write them in different methods
    """
    def operation(self):
        # RULE --> op: term ((GT|LT|GE|LE|EQUAL|CHOICE|PLUS|MINUS) term)*

        left_node = self.term()

        # Checks if left node has been received and if the following token is one of the following tokens: : =, <, >, <=, >=, |, +, -

        if(left_node.hasSyntaxError == False and (self.check_type(self.current_token.type,
                [TokenTypes.GREATER,TokenTypes.GREATEREQ,TokenTypes.LOWER,TokenTypes.LOWEREQ, TokenTypes.CHOICE, TokenTypes.PLUS,
                TokenTypes.MINUS, TokenTypes.EQUAL]))):

                node = ParsedNode(None,True)
                
                # The while method "concatenates" the operations

                while(self.check_type(self.current_token.type,
                [TokenTypes.GREATER,TokenTypes.GREATEREQ,TokenTypes.LOWER,TokenTypes.LOWEREQ, TokenTypes.CHOICE, TokenTypes.PLUS,
                TokenTypes.MINUS, TokenTypes.EQUAL])):
                
                    token = self.current_token
                    self.forward()
                    right_node = self.term()
                    if(right_node.hasSyntaxError):
                        return right_node
                    
                    # Binds found operation to its left node
                    if(node.node == None):
                       node = ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
                    else: node = ParsedNode(OperatorNode(token,node.node,right_node.node),False)
                return node
        return left_node

    """
    Checks the same way in operation method but here it checks for *, /
    """
    def term(self) -> ParsedNode:
        # RULE --> factor ((MUL|DIV) factor)*
        
        left_node = self.factor() 

        if(left_node.hasSyntaxError == False and (self.check_type(self.current_token.type,[TokenTypes.MULTIPLY, TokenTypes.DIVIDE]))):
            node = ParsedNode(None,True)

             # The while method "concatenates" the operations
            while(self.check_type(self.current_token.type,[TokenTypes.MULTIPLY, TokenTypes.DIVIDE])):
               
                token = self.current_token
                self.forward()
                right_node = self.factor()
                if(right_node.hasSyntaxError):
                    return right_node
                
                # Binds found operation to its left node
                if(node.node == None):
                  node = ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
                else: node = ParsedNode(OperatorNode(token,node.node,right_node.node),False)
            return node
        return left_node
    
    """
    Checks for unary operations, Integers, brackets (highest priority)
    """
    def factor(self) -> ParsedNode:
      
      # RULE -->  INTEGER  
       # : brackets
       # : (MINUS|PLUS) arith
       # : func_call x() x
       # : indexing     NOT IMPLEMENTING
       # : --> means the same as (brackets|unary|func_call) just like in operation()
       # only that for each if a different Node may be created not such as only OperationNode like in operation()
       
        token = self.current_token
        index = self.lexer.index

      
        #Integer check
        if(token.type == TokenTypes.INTEGER):
            self.forward()
            return ParsedNode(NumberNode(token),False)
        
        #Unary operation check
        if(self.check_type(self.current_token.type,[TokenTypes.PLUS, TokenTypes.MINUS])):
            self.forward()
            node = self.operation()
            if(node.hasSyntaxError):        # (--) --> Error needs (-- expr) or (--3)
                return ParsedNode(None, True)
            return ParsedNode(UnaryNode(token,node.node),False)
        
        #Brackets check
        return self.brackets() #Returns invalid Node on invalid Syntax
    
    """
    Checks for brackets (highest priority)
    """
    def brackets(self) -> ParsedNode:
        # RULE --> brackets: LB expr RB 
        if(self.current_token.type == TokenTypes.LBRACKET):
            self.forward()
            node = self.expr()
        
            if(self.current_token.type == TokenTypes.RBRACKET):
                self.forward()
                return node
        return ParsedNode(None,True)
        

    """
    METHODS FROM HERE NEEDS TO BE FULLY IMPLEMENTED AND CHECKED (except forward, check_type, set_to_token)!
    """

    """
    y := 8 y:=(x:int)  y:= method(...)...
    """
    def binding(self) -> ParsedNode:
          # RULE --> scope BINDING expr     NEED UPDATE
        left_node = self.scope()

        if(left_node.hasSyntaxError == False):
            if(self.check_type(self.current_token.type,[TokenTypes.BINDING])):
                token = self.current_token
                self.forward()
                right_node = self.expr()
                if(right_node.hasSyntaxError == False):
                    pass # Return Node
                else: ParsedNode(None,True)
        return left_node

    """
    x:int
    """
    def scope(self) -> ParsedNode:
      # RULE --> identifier COLON type    NEED UPDATE

        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.check_type(self.current_token.type,[TokenTypes.COLON])):
                token = self.current_token
                self.forward()
                type = self.type()
                if(type.hasSyntaxError == False):
                    pass # Return Scope Node
                else: ParsedNode(None,True)
        return left_node
    

    """
    variable/method name
    """
    def identifier(self) -> ParsedNode:
        #RULE --> identifier            NEED UPDATE

        token = self.current_token

        if(token.type == TokenTypes.IDENTIFIER):
            self.forward()
            pass # Return identifier Node
        return ParsedNode(None, True) 
        
    """
    int or tuple(int,int) or array{int}
    """
    def type(self) -> ParsedNode:
        # RULE -->  INT                         NEED UPDATE
        #        : TUPLE LB type (,type)* RB    

        token = self.current_token
        if(token == TokenTypes.INT_TYPE):
            self.forward()
            pass # Return Node
        
        if(token == TokenTypes.TUPLE_TYPE):
             self.forward()
             if(token == TokenTypes.LBRACKET):
                 
                 types:list[TypeNode] = []

                 types = self.type()
                 if(types.hasSyntaxError == False):
                     if(self.check_type(self.current_token.type, [TokenTypes.COMMA])):
                        while(self.current_token.type == TokenTypes.COMMA):

                            self.forward()
                            t = self.type()

                            if(t.hasSyntaxError):  #If on error
                                return ParsedNode(None,True)
                            types.append(t) #else append to list of types
                      
                     pass # Return TypNode with list of nodes

             ParsedNode(None, True) 
        
        return ParsedNode(None, True) 
        
    """
    a[i:int]
    """
    def indexing(self) -> ParsedNode:
        # RULE --> identifier SLB expr SRB

        left_node = self.identifier()

        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.SLB):
                self.forward()
                expr_node = self.expr()
          
                if(expr_node.hasSyntaxError == False and self.current_token == TokenTypes.SRB):
                     pass # RETURN INDEXING NODE
                return ParsedNode(None,True)
        return left_node
                

    """
    Moves forward in the tokens list
    """
    def forward(self) -> None:
        print(self.current_token.__info__())
        self.lexer.forward()
        self.current_token = lexer.get_token(self.lexer.current_char)
        if (self.current_token.type == TokenTypes.EOF):
            self.end = True

        
    """
    Checks if a type exists in the following types list
    """
    def check_type(self,type:TokenTypes,types:list[TokenTypes]) -> bool:
        return type in types
    

    """
    Sets current token back if a certain path lead to failure (Wrong syntax)
    May need it for later
    """
    def set_to_token(self,index): 
        self.lexer.index = index
        self.forward()


In [16]:
class Interpreter:
    def __init__(self, parser: Parser):
        self.parser = parser

    def interpret(self):
        tree = self.parser.parse()
        
        return tree.visit(tree)

In [17]:
text = "3 | 7 | 4"
lexer = lexicon(text)
parser = Parser(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
result

TokenTypes.INTEGER: 3
TokenTypes.CHOICE: |
TokenTypes.INTEGER: 7
TokenTypes.CHOICE: |
TokenTypes.INTEGER: 4


## Testing for statement

In [55]:
# text = "for { 1..10 }"
text = "1..20"
lexer = lexicon(text)
parser = Parser(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
result

TokenTypes.INTEGER: 1
TokenTypes.DOT: .
TokenTypes.DOT: .
TokenTypes.INTEGER: 20


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]